# Code to work with the website

This notebook showcases all processed with the website that Python can do. You need a Firebase Admin key to work with the Firestore database.

## Create array for the website of evaluation books

In [ ]:
import pandas as pd

In [ ]:
evaluation_books = pd.read_csv("../data/external/evaluation.csv")

In [ ]:
textfile = open("../data/interim/evaluation.txt", "w")
for element in evaluation_books.itemID.tolist():
    textfile.write(str(element) + ",")
textfile.close()

## Download Decision data

In [1]:
import pandas as pd
from src.data.firestore import connect_to_firestore, retrieve_documents_from_collection

In [2]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [3]:
decisions_dict = retrieve_documents_from_collection(firestore_client, collection="decisions")

In [7]:
decisions = pd.DataFrame(decisions_dict)

In [8]:
decisions.to_csv("../decisions.csv", index = False)

,team,recommendation_shown_2,recommendation_shown_1,book_id,alias,model_1_id,decision,model_0_id,time,recommendation_shown_0,model_2_id
0,MannheimMiners,69193,62101,77554,DP,model_2,recommendation_1,team404_base_model,2021-06-11 11:24:00.379000+00:00,39220,teamfivemodel
1,The Dataworms,3286,8732,25488,Mimimi,team404_base_model,recommendation_1,Hybrid_MeanAllClassifiers,2021-06-12 13:04:27.267000+00:00,9911,model_2
2,DataMinerz,41819,22378,31827,Jakob,Hybrid_MeanAllClassifiers,recommendation_1,team404_base_model,2021-06-08 09:51:03.521000+00:00,18009,teamfivemodel
3,The Dataworms,17252,17252,58236,Keyvan Amiri Elyasi,model_2,recommendation_2,Hybrid_MeanAllClassifiers,2021-06-13 14:07:45.203000+00:00,58236,first
4,D2G2,18048,14903,13970,seb,first,recommendation_2,teamfivemodel,2021-06-10 07:30:03.276000+00:00,23529,Hybrid_MeanAllClassifiers
...,...,...,...,...,...,...,...,...,...,...,...
6308,The Dataworms,43537,69203,1460,Keyvan Amiri Elyasi,Hybrid_MeanAllClassifiers,recommendation_1,model_2,2021-06-13 09:49:02.868000+00:00,9211,first
6309,Dataworms,32762,70706,77956,Nazanin,model_2,recommendation_2,team404_base_model,2021-06-13 08:53:22.354000+00:00,33972,first
6310,flying chicken wings,60471,51370,13382,Guszti,teamfivemodel,recommendation_0,team404_base_model,2021-06-12 09:22:04.701000+00:00,69023,model_2
6311,MannheimMiners,1041,1041,8542,bechtify,teamfivemodel,recommendation_0,first,2021-06-11 11:57:13.815000+00:00,1041,model_2


## Upload book information to the website

In [ ]:
import pandas as pd
from src.data.firestore import connect_to_firestore, write_documents_to_collection

In [ ]:
books_path = "../data/processed/books.csv"
books_reduced_path = "../data/processed/books_reduced.csv"

In [ ]:
books = pd.read_csv(books_path,index_col = 0)
books_reduced_index = pd.read_csv(books_reduced_path,index_col = 0)

In [ ]:
books.set_index("itemID", inplace = True)

In [ ]:
books_reduced = books.filter(books_reduced_index.book_id.tolist(), axis = 0)

In [ ]:
books_reduced.reset_index(inplace = True)

In [ ]:
books_reduced

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

Since we have a free tier on firebase we need to upload multiple days. In moment we have the first 10.000 online.

In [ ]:
write_documents_to_collection(firestore_client, books_reduced, collection="books")

## Upload a new model to the website

If a team sends us a new model, we can upload it to the website with a simple function. Before we upload the recommendations, we have to create document in the model_scores collection for the new model. This requires to use the firebase backend.

In [ ]:
import pandas as pd
from src.data.firestore import connect_to_firestore, write_documents_to_collection

Code for Team DataMinerz

In [ ]:
DataMinerz_path = "../data/processed/recommendations.csv"

In [ ]:
DataMinerz_recommendations = pd.read_csv(DataMinerz_path,index_col = 0)

In [ ]:
DataMinerz_recommendations

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, DataMinerz_recommendations, collection="recommendations")

Code for Team MannheimMiners

In [ ]:
mannheimminers_path = "../data/processed/MannheimMiners.csv"

In [ ]:
mannheimminers_recommendations = pd.read_csv(mannheimminers_path,index_col = 0)

In [ ]:
mannheimminers_recommendations.dropna(inplace=True)

In [ ]:
mannheimminers_processed = mannheimminers_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
mannheimminers_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, mannheimminers_processed, collection="recommendations")

Code for Team team404

In [ ]:
team404_path = "../data/processed/Team_02.xlsx"

In [ ]:
team404_recommendations = pd.read_excel(team404_path,index_col = 0)

In [ ]:
team404_recommendations.dropna(inplace=True)

In [ ]:
team404_recommendations.reset_index(inplace = True)

In [ ]:
team404_processed = team404_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
team404_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, team404_processed, collection="recommendations")

Code for Team minefromhome

In [ ]:
minefromhome_path = "../data/processed/Team_05.csv"

In [ ]:
minefromhome_recommendations = pd.read_csv(minefromhome_path,index_col = 0, names = ["book_id",
                                                                                     "model_id",
                                                                                     "team_id",
                                                                                     "recommendation_1",
                                                                                     "recommendation_2",
                                                                                     "recommendation_3",
                                                                                     "recommendation_4",
                                                                                     "recommendation_5"])

In [ ]:
minefromhome_recommendations

In [ ]:
minefromhome_recommendations.dropna(inplace=True)

In [ ]:
minefromhome_recommendations.reset_index(inplace = True)

In [ ]:
minefromhome_processed = minefromhome_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
minefromhome_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, minefromhome_processed, collection="recommendations")

Code for Team flying_chicken_wings

In [ ]:
flying_chicken_wings_path = "../data/processed/Team_06.xlsx"

In [ ]:
flying_chicken_wings_recommendations = pd.read_excel(flying_chicken_wings_path)

In [ ]:
flying_chicken_wings_recommendations.dropna(inplace=True)

In [ ]:
flying_chicken_wings_processed = flying_chicken_wings_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
flying_chicken_wings_processed

In [ ]:
"team404_base_model", "teamfivemodel", "Hybrid_MeanAllClassifiers"]

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, flying_chicken_wings_processed, collection="recommendations")

Code for Team flying_chicken_wings

In [ ]:
flying_chicken_wings_path = "../data/processed/Team_06.xlsx"

In [ ]:
flying_chicken_wings_recommendations = pd.read_excel(flying_chicken_wings_path)

In [ ]:
flying_chicken_wings_recommendations.dropna(inplace=True)

In [ ]:
flying_chicken_wings_processed = flying_chicken_wings_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
flying_chicken_wings_processed

In [ ]:
"team404_base_model", "teamfivemodel", "Hybrid_MeanAllClassifiers"]

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, flying_chicken_wings_processed, collection="recommendations")

Code for d2g2

In [ ]:
d2g2_path = "../data/processed/hybrid_linear_bpe_graph.csv"

In [ ]:
d2g2_recommendations = pd.read_csv(d2g2_path)

In [ ]:
d2g2_recommendations

In [ ]:
d2g2_recommendations.dropna(inplace=True)

In [ ]:
d2g2_processed = d2g2_recommendations.astype({"book_id": int, "model_id": str,
                                                                        "team_id": str, "recommendation_1":int,
                                                                        "recommendation_2":int,
                                                                        "recommendation_3":int, 
                                                                        "recommendation_4":int,
                                                                        "recommendation_5":int})

In [ ]:
d2g2_processed

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, d2g2_processed, collection="recommendations")

## Update book information to the website

In [ ]:
import pandas as pd
import pickle
from src.data.firestore import connect_to_firestore, write_documents_to_collection

In [ ]:
books_path = "../data/processed/books.csv"
books_reduced_path = "../data/processed/books_reduced.pkl"

In [ ]:
books = pd.read_csv(books_path,index_col = 0)
books_reduced_index = pd.read_pickle(books_reduced_path)

In [ ]:
books_added_index = set(pd.melt(d2g2_processed, value_vars=['recommendation_1',
                                       'recommendation_2',
                                       'recommendation_3',
                                       'recommendation_4',
                                       'recommendation_5']).value.tolist())

In [ ]:
len(books_reduced_index)

In [ ]:
len(books_added_index)

In [ ]:
books_reduced_added_index = {index for index in books_added_index if index not in books_reduced_index}

In [ ]:
len(books_reduced_added_index)

In [ ]:
books.set_index("itemID", inplace = True)

In [ ]:
books_reduced_added = books.filter(books_reduced_added_index, axis = 0)

In [ ]:
books_reduced_added.reset_index(inplace = True)

In [ ]:
books_reduced_added

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
write_documents_to_collection(firestore_client, books_reduced_added, collection="books")

In [ ]:
len(books_reduced_index.union(books_reduced_added_index))

In [ ]:
with open('../data/processed/books_reduced.pkl', 'wb') as file:
    pickle.dump(books_reduced_index.union(books_reduced_added_index), file)

## Delete error data

In [ ]:
from src.data.firestore import connect_to_firestore, write_documents_to_collection

In [ ]:
firestore_client = connect_to_firestore(credentials="../secrets/dmc-book-recommendation-firebase-adminsdk-yebqi-d805561028.json",
                         database_url="https://book-recommendation-website.firebaseio.com/")

In [ ]:
batch = firestore_client.batch()

doc_ref = firestore_client.collection('model_scores').where('score', '!=', False).get()

for doc in doc_ref:
       batch.delete(doc.reference)

batch.commit()

In [ ]:
float('nan')